In [ ]:
import matplotlib
import numpy
import json
import dataclasses

@dataclasses.dataclass
class Answer:
	eye_shape : str
	eyebrow : str
	iris : str
	pupil : str

@dataclasses.dataclass
class Data:
	id : str
	answers : list[Answer]



def load_log_raw(path):
	with open(path, "r") as f:
		return json.load(f)
	

def load_log(path):
	raw = load_log_raw(path)
	ret = Data(0, [])
	for entry in raw:
		entry_type = entry['type']
		if entry_type == 'user_id':
			ret.id = entry['id']
		if entry_type == 'submit_answer':
			ret.answers.append(Answer(
				entry['eye_shape'],
				entry['eyebrow'],
				entry['iris'],
				entry['pupil'],
			))
	return ret


In [ ]:
load_log("./data/1.log").answers

14